# RLlib Sample Application: CartPole-v0

We were briefly introduced to the _CartPole_ example and the OpenAI gym `CartPole-v0` environment ([gym.openai.com/envs/CartPole-v0/](https://gym.openai.com/envs/CartPole-v0/)) in the [introduction](01-Introduction-to-Reinforcement-Learning.ipynb). This lesson uses [RLlib](https://ray.readthedocs.io/en/latest/rllib.html) to train a policy for _CartPole_.

Recall that the `gym` Python module provides MDP interfaces to a variety of simulators, like the simple simulator for the physics of balancing a pole on a cart that is used by the CartPole environment. The _CartPole_ problem is described at https://gym.openai.com/envs/CartPole-v0. Here is an image from that website:

![Cart Pole](../images/Cart-Pole.png)

Even though this is a relatively simple and quick example to run, its results can be understood quite visually.

For more background about this problem, see:

* ["Neuronlike Adaptive Elements That Can Solve Difficult Learning Control Problem"](https://ieeexplore.ieee.org/document/6313077), AG Barto, RS Sutton, and CW Anderson, *IEEE Transactions on Systems, Man, and Cybernetics* (1983)
* ["Cartpole - Introduction to Reinforcement Learning (DQN - Deep Q-Learning)"](https://towardsdatascience.com/cartpole-introduction-to-reinforcement-learning-ed0eb5b58288), [Greg Surma](https://twitter.com/GSurma)

First, import Ray and the PPO module in RLlib, then start Ray.

In [ ]:
import ray
import ray.rllib.agents.ppo as ppo

In [ ]:
import pandas as pd
import json, os, shutil

Model *checkpoints* will get saved after each iteration into directories under `/tmp/ppo/cart`.

> **Note:** If you prefer to use a different directory root than `/tmp`, change it in the next cell **and** in the `rllib rollout` cell below.

In [ ]:
checkpoint_root = '/tmp/ppo/cart'

Clean up output of previous lessons (optional):

In [ ]:
# Where checkpoints are written:
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)
# Where some data will be written and used by Tensorboard below:
shutil.rmtree(f'{os.getenv("HOME")}/ray_results/', ignore_errors=True, onerror=None)

In [ ]:
ray.init(ignore_reinit_error=True)

The Ray Dashboard is useful for monitoring Ray:

In [ ]:
print(f'Dashboard URL: http://{ray.get_webui_url()}')

Next we'll train an RLlib policy with the [`CartPole-v0` environment](https://gym.openai.com/envs/CartPole-v0/).

By default, training runs for `10` iterations. Increase the `N_ITER` setting if you want to train longer and see the resulting rewards improve.

> **Note:** If you change the values shown for `config['model']['fcnet_hiddens']`, make the same change in the `rllib rollout` command below!

In [ ]:
SELECT_ENV = "CartPole-v0"
N_ITER = 10

config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"
# Other settings we might use:
config['num_workers'] = 1
config['num_sgd_iter'] = 10
config['sgd_minibatch_size'] = 512
config['model']['fcnet_hiddens'] = [100, 50]
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed

In [ ]:
agent = ppo.PPOTrainer(config, env=SELECT_ENV)

results = []
episode_data = []
episode_json = []
for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    episode = {'n': n, 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']}
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)
    print(f'Max reward: {episode["episode_reward_max"]}. Checkpoint saved to {file_name}')

The episode rewards should increase after multiple iterations. Try tweaking the config parameters. Smaller values for the `num_sgd_iter`, `sgd_minibatch_size`, or the `model`'s `fcnet_hiddens` will train faster, but take longer to improve the policy.

In [ ]:
df = pd.DataFrame(data=episode_data)
df

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
import bokeh.io
# The next two lines prevent Bokeh from opening the graph in a new window.
bokeh.io.reset_output()
bokeh.io.output_notebook()

In [ ]:
source = ColumnDataSource(df)

plot = figure(title='Episode reward and length means/maxes')
plot.grid.grid_line_alpha=0.2
plot.xaxis.axis_label = 'n'
plot.yaxis.axis_label = 'value'

plot.line(x='n', y='episode_reward_mean', source=source, color='blue', legend_label='Episode reward mean', name='Episode reward mean')
plot.circle(x='n', y='episode_reward_mean', source=source, color='blue', size=8)
plot.line(x='n', y='episode_reward_max', source=source, color='green', legend_label='Episode reward max', name='Episode reward max')
plot.circle(x='n', y='episode_reward_max', source=source, color='green', size=8)
plot.legend.location = "top_left"

hover = HoverTool()
hover.tooltips = [
    ("n", "$x"),
    ("mean", "$y")]
plot.add_tools(hover)

show(plot)

Also, print out the policy and model to see the results of training in detail…

In [ ]:
import pprint

policy = agent.get_policy()
model = policy.model

pprint.pprint(model.variables())
pprint.pprint(model.value_function())

print(model.base_model.summary())

Next we'll use the [`rollout` script](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies), using the `rllib rollout` command line, to evaluate the trained policy.

This visualizes the "cartpole" agent operating within the simulation: moving the cart left or right to avoid having the pole fall over.

> **Notes:** 
>
> 1. If you defined `checkpoint_root` above to be different than `/tmp/ppo/cart`, then change it here, too. Note that you can't use variable substitution in the next cell, unfortunately.
> 2. If you changed the model parameters, specifically the `fcnet_hiddens` array in the `config` object above, make the same change here.

In [ ]:
! rllib rollout \
    /tmp/ppo/cart/checkpoint_2/checkpoint-2 \
    --config "{\"env\": \"CartPole-v0\", \"model\": {\"fcnet_hiddens\": [100, 50]}}" \
    --run PPO \
    --steps 2000

The rollout uses the second saved checkpoint, `checkpoint_2`, evaluated through `2000` steps.
Modify the path to view other checkpoints. Note that you have to change the number in _two_ places.

Finally, launch [TensorBoard](https://ray.readthedocs.io/en/latest/rllib-training.html#getting-started) then follow the instructions (for example, click the URL link or copy and paste it into a browser) to visualize key metrics from training with RLlib…


In [ ]:
!tensorboard --logdir=~/ray_results/

Next, go through any of the following lessons:

* [04a: Application: Mountain Car](04a-Application-Mountain-Car.ipynb) -- Based on the `MountainCar-v0` environment from OpenAI Gym.
* [04b: Application: Taxi](04b-Application-Taxi.ipynb) -- Based on the `Taxi-v3` environment from OpenAI Gym.
* [04c: Application: Frozen Lake](04c-Application-Frozen-Lake.ipynb) -- Based on the `FrozenLake-v0` environment from OpenAI Gym.